In [130]:
import os
from skimage import io, transform
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import math

In [131]:
class SequenceDriveDataset(Dataset):

    def __init__(self, csv_file, root_dir, bags, time_steps, transform=None):
        self._csv_file = csv_file
        self._root_dir = root_dir
        self._bags = bags
        self._steps = time_steps + 1 # +1 to adjust for indexing
        self._transform = transform
        self._frames = self.get_frames()

    def __len__(self):
        return len(self._frames)

    def __getitem__(self, idx):
        images = []
        for step in range(1,self._steps):
            img_name = os.path.join(
                self._root_dir, self._frames['filename_L{}'.format(step)].iloc[idx]
            )
            images.append(io.imread(img_name))
            
        target = self._frames['angle'].iloc[idx]
        sample = {'image': images, 'steer': target}

        if self._transform:
            pass
            #sample = self._transform(sample)

        return sample

    def get_frames(self):
        file_paths = [
            (bag,os.path.join(self._root_dir, bag, self._csv_file))
            for bag in self._bags
        ]

        frames = []
        unused = ['width', 'height','lat','long','alt','timestamp']
        for bag,path in file_paths:
            df = pd.read_csv(path)
            # Add bag column and generate correct paths.
            df['bag'] = bag
            df.filename = df.bag + '/'+ df.filename
            # Remove unused to columns.
            for col in unused:
                del df[col]
                
            frames.append(df[df.frame_id=='center_camera'])
            
        all_df = pd.concat(frames, axis=0, ignore_index=True)
        
        lags_df = (pd.concat([all_df.filename.shift(i) for i in range(self._steps)], axis=1,
               keys=['filename'] + ['filename_L%s' % i for i in range(1, self._steps)]).dropna())
        # To ensure that only past frames are used to predict current steering.
        # (e.g to predict target at t, we use t-1,t-2, etc.)
        lags_df = lags_df.drop(['filename'], axis=1)
        final_df = pd.merge(all_df, lags_df, left_index=True, right_index=True)
        return final_df



In [132]:
bags = ['bag4','bag5']
root_dir = r'/home/tasuku/workspace/dl/algorithms/4-1-nvidia_end_to_end_self_driving/data/output'
csv_file = r'interpolated.csv'

In [133]:
data = SequenceDriveDataset(csv_file, root_dir, bags, time_steps=10)

# Test if get_frames is generating correct data.
df = data.get_frames()
df.head()

,index,frame_id,filename,angle,torque,speed,bag,filename_L1,filename_L2,filename_L3,filename_L4,filename_L5,filename_L6,filename_L7,filename_L8,filename_L9,filename_L10
10,2016-11-17 23:35:30.331103013,center_camera,bag4/center/1479425730331103013.png,-0.015708,-0.332230,22.325147,bag4,bag4/center/1479425730281174632.png,bag4/center/1479425730231104826.png,bag4/center/1479425730181075491.png,bag4/center/1479425730131106872.png,bag4/center/1479425730081148517.png,bag4/center/1479425730031118702.png,bag4/center/1479425729981076760.png,bag4/center/1479425729931106688.png,bag4/center/1479425729881120069.png,bag4/center/1479425729831388501.png
11,2016-11-17 23:35:30.381154894,center_camera,bag4/center/1479425730381154894.png,-0.015708,-0.300729,22.325001,bag4,bag4/center/1479425730331103013.png,bag4/center/1479425730281174632.png,bag4/center/1479425730231104826.png,bag4/center/1479425730181075491.png,bag4/center/1479425730131106872.png,bag4/center/1479425730081148517.png,bag4/center/1479425730031118702.png,bag4/center/1479425729981076760.png,bag4/center/1479425729931106688.png,bag4/center/1479425729881120069.png
12,2016-11-17 23:35:30.431228626,center_camera,bag4/center/1479425730431228626.png,-0.015708,-0.023482,22.308510,bag4,bag4/center/1479425730381154894.png,bag4/center/1479425730331103013.png,bag4/center/1479425730281174632.png,bag4/center/1479425730231104826.png,bag4/center/1479425730181075491.png,bag4/center/1479425730131106872.png,bag4/center/1479425730081148517.png,bag4/center/1479425730031118702.png,bag4/center/1479425729981076760.png,bag4/center/1479425729931106688.png
13,2016-11-17 23:35:30.481172304,center_camera,bag4/center/1479425730481172304.png,-0.012561,0.162913,22.298315,bag4,bag4/center/1479425730431228626.png,bag4/center/1479425730381154894.png,bag4/center/1479425730331103013.png,bag4/center/1479425730281174632.png,bag4/center/1479425730231104826.png,bag4/center/1479425730181075491.png,bag4/center/1479425730131106872.png,bag4/center/1479425730081148517.png,bag4/center/1479425730031118702.png,bag4/center/1479425729981076760.png
14,2016-11-17 23:35:30.531147589,center_camera,bag4/center/1479425730531147589.png,-0.007674,0.306556,22.278042,bag4,bag4/center/1479425730481172304.png,bag4/center/1479425730431228626.png,bag4/center/1479425730381154894.png,bag4/center/1479425730331103013.png,bag4/center/1479425730281174632.png,bag4/center/1479425730231104826.png,bag4/center/1479425730181075491.png,bag4/center/1479425730131106872.png,bag4/center/1479425730081148517.png,bag4/center/1479425730031118702.png


In [134]:
# Test getitem and see if shape is desired shape.
np.array(data[1]['image']).shape

(10, 480, 640, 3)